In [1]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [2]:

# -----------------------------
# Preparación de datos
# -----------------------------

df = pd.read_csv('Caracteristicas_final.csv')

X = df.drop(columns=['is_fraud', 'trans_date_trans_time', 'dob', 'first', 'last', 'street', 'city', 'state', 'job', 'merchant', 'category'])  # excluye variables irrelevantes o no numéricas
y = df['is_fraud']

# Dividir el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)


In [3]:
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()
print("Columnas categóricas:", categorical_cols)

Columnas categóricas: ['gender', 'trans_num', 'trans_day_date']


In [4]:
from sklearn.preprocessing import LabelEncoder

# Eliminar columnas con IDs únicos antes de codificar
categorical_cols = [col for col in categorical_cols if col != 'trans_num']


encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    encoders[col] = le

    # Mapear X_test usando el encoder, asignando un valor -1 para valores no vistos
    X_test[col] = X_test[col].astype(str).map(lambda s: le.transform([s])[0] if s in le.classes_ else -1)


### Entrenar modelo base

In [5]:
model = lgb.LGBMClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: trans_num: object